In [ ]:
!pip install -U -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_

In [ ]:
# prompt: How to access secret key saved in the secret key section and use it for the login of hugging face automatically

from google.colab import userdata
import huggingface_hub

huggingface_token = userdata.get('HF_TOKEN')

# This will log you in using the token without requiring interactive input
huggingface_hub.login(token=huggingface_token)


# Step 2 : Importing the Dataset and Preprocessing
The dataset consists of conversions between a customer care sales agent and a customer. Typically at max they have 10 from each.

In [ ]:
from datasets import load_dataset
import re

dataset = load_dataset('philschmid/gretel-synthetic-text-to-sql',split='train')


README.md:   0%|          | 0.00/737 [00:00<?, ?B/s]

(…)nthetic_text_to_sql_train.snappy.parquet:   0%|          | 0.00/32.4M [00:00<?, ?B/s]

(…)ynthetic_text_to_sql_test.snappy.parquet:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
    num_rows: 100000
})

In [ ]:
dataset[0]

{'id': 5097,
 'domain': 'forestry',
 'domain_description': 'Comprehensive data on sustainable forest management, timber production, wildlife habitat, and carbon sequestration in forestry.',
 'sql_complexity': 'single join',
 'sql_complexity_description': 'only one join (specify inner, outer, cross)',
 'sql_task_type': 'analytics and reporting',
 'sql_task_type_description': 'generating reports, dashboards, and analytical insights',
 'sql_prompt': 'What is the total volume of timber sold by each salesperson, sorted by salesperson?',
 'sql_context': "CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');",
 'sql'

For llama 2 model there is a prompt template for single instance message


```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Who are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
```



# Step 3 : Feature Selection
Out all features available in dataset the following features are using \\
Features



*   sql_context : sql commands of the database , explain schema of the database
*   sql_prompt : user question
*   sql : Query to be generated.



In [ ]:
# prompt: reduce the number of rows in the dataset by deleting them and limit them to only 2000

dataset = dataset.select(range(500))

dataset

Dataset({
    features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
    num_rows: 500
})

In [ ]:

# System message for the assistant
system_message = f"""You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA."""

# User prompt that combines the user query and the schema
user_message = """Given the <USER_TASK> and the <SCHEMA>, generate the corresponding SQL command to retrieve the desired data, considering the query's syntax, semantics, and schema constraints.

<SCHEMA>
{context}
</SCHEMA>

<USER_TASK>
{question}
</USER_TASK>
"""

assistant = """{result}"""
prompt_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system_message}<|eot_id|><|start_header_id|>user<|end_header_id|>

{user_message}<|eot_id|><|start_header_id|>assistant<|end_header_id|>{assistant}<|eot_id|>"""


def create_conversation(sample):
  prompt =  prompt_template.format(context=sample['sql_context'], question=sample['sql_prompt'], result =sample['sql'])
  return {
      'message': prompt,
  }

# Convert dataset to OAI messages
dataset = dataset.map(create_conversation, remove_columns=dataset.features,batched=False)


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['message'],
    num_rows: 500
})

# Step 3 : Training Arguments and Model Loading

In [ ]:
dataset['message'][0]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the <USER_TASK> and the <SCHEMA>, generate the corresponding SQL command to retrieve the desired data, considering the query's syntax, semantics, and schema constraints.\n\n<SCHEMA>\nCREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');\n</SCHEMA>\n\n<USER_TASK>\nWhat is the total volume of timber sold by each salesperson, sorted by salesperson?\n<

In [ ]:
dataset = dataset.select(range(100))

In [ ]:
from transformers import AutoTokenizer
model_id = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [ ]:
tokenized_data = dataset.map(lambda x: tokenizer(x['message']), batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
tokenized_data

Dataset({
    features: ['message', 'input_ids', 'attention_mask'],
    num_rows: 100
})

In [ ]:
tokenized_data.save_to_disk("/content/drive/MyDrive/Trail Hugging Face/dataset/")

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# prompt: read the dataset from the drive location "/content/drive/MyDrive/Trail Hugging Face/dataset/"

from datasets import load_from_disk

dataset = load_from_disk("/content/drive/MyDrive/Trail Hugging Face/dataset/")
dataset

Dataset({
    features: ['message', 'input_ids', 'attention_mask'],
    num_rows: 100
})

In [ ]:
dataset = dataset.select(range(10))

In [ ]:
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.4 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import  AutoModelForCausalLM, BitsAndBytesConfig

# Hugging Face model id
model_id = "meta-llama/Llama-3.2-1B"

# BitsAndBytesConfig: Enables 4-bit quantization to reduce model size/memory usage
bitconfig= BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16
)

# Load tokenizer and model with QLoRA configuration
torch_dtype = torch.float16


# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id, device_map= {"":0,},config = bitconfig)

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig

peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=4,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM",
    modules_to_save=["lm_head", "embed_tokens"]
)

In [ ]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [ ]:
from trl import SFTConfig

args = SFTConfig(
    output_dir="llama3.2-text-to-sql",
    max_seq_length=128,
    packing=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    logging_steps=10,
    fp16 = True,
    bf16 = False,
    save_strategy="epoch",
    learning_rate=2e-4,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    push_to_hub=False,
    report_to="none",
    dataset_text_field="message", # Specify the text field in the dataset
)

average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.


In [ ]:
# prompt: change trainer object to use the pre tokenized data instead of again tokenizing at the training stage and also change args to update the changes

from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_data, # Use the pre-tokenized dataset
    peft_config=peft_params,
    args=args,
    tokenizer=tokenizer,
    data_collator=None, # No data collator needed since data is pre-tokenized
    # dataset_text_field="message", # This is no longer needed since we are using tokenized data
)

# Update the training arguments
args.dataset_text_field = None  # Set the text field to None since we use tokenized data
args.packing = False  # Disable packing if using pre-tokenized data unless you've manually packed it

trainer


In [ ]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    args = args)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:404: UserWarning: Padding-free training is enabled, but the attention implementation is not set to 'flash_attention_2'. Padding-free training flattens batches into a single sequence, and 'flash_attention_2' is the only known attention mechanism that reliably supports this. Using other implementations may lead to unexpected behavior. To ensure compatibility, set `attn_implementation='flash_attention_2'` in the model configuration, or verify that your attention mechanism can handle flattened sequences.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:450: UserWarning: You are using packing, but the attention implementation is not set to 'flash_attention_2'. Packing flattens batches into a single sequence, and 'flash_attention_2' is the only known attention mechanism that reliably supports this. Using other implementations may lead to cross-contamination between batches. To avoid this, ei

Packing train dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss


TrainOutput(global_step=1, training_loss=3.524115562438965, metrics={'train_runtime': 99.9319, 'train_samples_per_second': 0.1, 'train_steps_per_second': 0.01, 'total_flos': 9512697200640.0, 'train_loss': 3.524115562438965})

In [ ]:
trainer.model.save_pretrained('./content/drive/MyDrive/Trail Hugging Face/llama3.2-text-to-sql/model')
trainer.tokenizer.save_pretrained('./content/drive/MyDrive/Trail Hugging Face/llama3.2-text-to-sql/tokenizer')


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


('./content/drive/MyDrive/Trail Hugging Face/llama3.2-text-to-sql/tokenizer/tokenizer_config.json',
 './content/drive/MyDrive/Trail Hugging Face/llama3.2-text-to-sql/tokenizer/special_tokens_map.json',
 './content/drive/MyDrive/Trail Hugging Face/llama3.2-text-to-sql/tokenizer/tokenizer.json')

In [ ]:
trainer.model.save_pretrained('/content/drive/MyDrive/Trail Hugging Face/Fine Tuned Model/model')
trainer.tokenizer.save_pretrained('/content/drive/MyDrive/Trail Hugging Face/Fine Tuned Model/tokenizer')

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


('/content/drive/MyDrive/Trail Hugging Face/Fine Tuned Model/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Trail Hugging Face/Fine Tuned Model/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Trail Hugging Face/Fine Tuned Model/tokenizer/tokenizer.json')

In [ ]:
# prompt: download all files from llama3.2-text-to-sql to local computer

import shutil
import os
from google.colab import files

# Define the directory you want to download
directory_to_dowload = "llama3.2-text-to-sql"

# Create a zip archive of the directory
shutil.make_archive(directory_to_download, 'zip', directory_to_download)

# Download the zip file
files.download(f'{directory_to_download}_2.zip')

In [ ]:
# prompt: generate the zip file llama3.2-text-to-sql in /content/drive/MyDrive/Trail Hugging Face

# Define the directory you want to zip
directory_to_zip = "llama3.2-text-to-sql"

# Define the output path for the zip file
output_zip_path = "/content/drive/MyDrive/Trail Hugging Face/llama3.2-text-to-sql"

# Create a zip archive of the directory
shutil.make_archive(output_zip_path, 'zip', directory_to_zip)

print(f"Zip file created at: {output_zip_path}.zip")

Zip file created at: /content/drive/MyDrive/Trail Hugging Face/llama3.2-text-to-sql.zip
